In [ ]:
from scrapping import get_data
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from time import sleep

In [ ]:
db = pd.read_parquet('db.parquet')
winners = db.iloc[:, 1:7].set_index(['Sorteos'])
df_0 = pd.DataFrame(columns = [str(i) for i in range(1, 51)], index = [0]).fillna(True)
winners = pd.concat([df_0, winners], ignore_index = True).reset_index(drop = True)

for e in range(1, 6):
    for i in range(1, 51):
        winners.loc[winners[f"Nro{e}"] == i, f"{i}"] = True

winners.drop(columns = ["Nro1","Nro2","Nro3","Nro4","Nro5"], inplace = True)
winners.drop(columns = winners.iloc[:, 50:100], inplace = True)

In [ ]:
draws = list(np.arange(1, len(winners)))
c = [str(i) for i in range(1, 51)]
dicts = {draw: {key:[] for key in c} for draw in draws}
contador = 0
e = 0

while e != len(winners) - 1:
    for columns in winners:
        df = winners.loc[:draws[e]]
        for i in reversed(df[columns]):
            if i is not True:
                contador += 1
            else:
                dicts[draws[e]].update({columns:contador})
                contador = 0
                break
    e += 1

skip_numbers = pd.DataFrame.from_dict(dicts, orient = 'index')

In [ ]:
last_draw = [t[::-1] for t in sorted(dicts[len(skip_numbers)].items(), key = lambda x: x[1])]
last_draw = [(a, int(b)) for a, b in last_draw]
ld_skips = pd.DataFrame(last_draw).groupby(0)[1].apply(list).reset_index().rename({0: 'Skip', 1: 'Numbers'}, axis = 1)
x_12 = np.arange(len(skip_numbers) - 12, len(skip_numbers) + 1)
sk_12 = skip_numbers.loc[x_12]
skips = np.arange(0, 19)
aus = []
last_7 = []
last_12 = []

for i in x_12[1:13]:
    columns = range(1, 51)
    for column in columns:
        if sk_12.loc[i, str(column)] == 0:
            aus.append(sk_12.loc[i - 1, str(column)])

for number in skips:
    last_12.append(aus.count(number))
    last_7.append(aus[25:60].count(number))
    
skips_7_12 = pd.DataFrame(np.column_stack([last_7, last_12]), columns = [7, 12])

In [ ]:
print(aus)

In [ ]:
aus[25:60]

In [ ]:
ld_skips.loc[2, 'Numbers'][0]

In [ ]:
# values = list(set(val for dic in [ultimo_sorteo] for val in dic.values()))